In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
from PIL import Image, ImageDraw
import os
from tqdm import tqdm
pd.options.display.max_rows = 100
pd.options.display.max_columns = 100

In [2]:
actual_st = pd.read_parquet("./stations/actual_stations.parquet")
actual_st['sis'] = actual_st['sis'].astype(int)
all_st = pd.read_parquet("./stations/all_stations.parquet")
all_st['sis'] = all_st['sis'].astype(int)
df_merge = pd.merge(actual_st, all_st, on='sis')
df_merge['lon_y'] = df_merge['lon_y'].replace(',', '.', regex=True).astype('float')
# actual_st['lon_e'] = actual_st['lon_y'].apply(lambda x: x if x>0 else x+360)
df_merge['lat_y'] = df_merge['lat_y'].replace(',', '.', regex=True).astype('float')
df_merge['diff_lat'] = df_merge['lat_x'] - df_merge['lat_y']
df_merge['diff_lon'] = df_merge['lon_x'] - df_merge['lon_y']
# actual_st = actual_st[actual_st.columns.to_list()[]]
actual_st = actual_st.sort_values(by=['lat'], ascending=False)
df_merge = df_merge.sort_values(by=['lat_x'], ascending=False)
actual_st.reset_index(drop=True, inplace=True)
actual_st.head()

,name,sis,lat,lon
0,Фуглойкалвен фьорд,1034,70.31700,20.15000
1,Торсваг фьорд,1033,70.24556,19.50028
2,Верхнедвинск,26554,55.82083,27.94000
3,Полоцк,26653,55.47472,28.75167
4,Шарковщина,26643,55.36306,27.45667


In [3]:
df_merge = df_merge[['name_x', 'sis', 'lat_x', 'lon_y', 'name_y', 'country']]
df_merge.columns = ['name', 'sis', 'lat', 'lon', 'name_en', 'country']
df_merge.head()


,name,sis,lat,lon,name_en,country
2,Фуглойкалвен фьорд,1034,70.31700,20.2,Fugloykalven Fyr,Norway
1,Торсваг фьорд,1033,70.24556,19.5,Torsvag Fyr,Norway
35,Верхнедвинск,26554,55.82083,27.9,Верхнедвинск,Беларусь
38,Полоцк,26653,55.47472,28.8,Полоцк,Беларусь
34,Шарковщина,26643,55.36306,27.5,Шарковщина,Беларусь


In [4]:
actual_st = df_merge

In [5]:
# actual_st

In [6]:
# lon = actual_st['lon'].values
# lat = actual_st['lat'].values
# lon[:2], lat[:2]

In [7]:
# lon.max(), lon.mean(), lon.min(), (lon.max()+lon.min())/2

In [8]:
# margin = 0
# FIGDPI = 100
# lons_min = lon.min() - margin
# lons_max = lon.max() + margin if lon.max() < 180 else lon.max()
# lats_min = lat.min() - margin
# lats_max = lat.max() + margin
# lons_min, lons_max, lats_min, lats_max

In [9]:
# TODO Rename this here and in `draw_map`
def _extracted_from_draw_map_19(m, 
                                projection, 
                                zoom,
                                lat,
                                lon,
                                tile,
                                slice_tile,
                                img_write=False,
                                plot_scatter=False,
                                names=[]
                                ):
        plt.rcParams['figure.dpi'] = 100
        # # plt.rcParams['figure.figsize'] = (10.24, 10.24)
        # plt.rcParams['figure.figsize'] = (5.12 * zoom, 5.12 * zoom)
        plt.rcParams['figure.figsize'] = (12,12)
        # plt.subplots_adjust(top = 1, bottom = 0, right = 1, left = 0, 
        #     hspace = 0, wspace = 0)
        # plt.margins(0,0)
        plt.axis('off')
        # m.drawcoastlines(linewidth=0.5, color='blue')
        # m.fillcontinents(color='gray', lake_color='aqua')
        # m.drawcountries(linewidth=1, color='red')

        # m.drawmapboundary(fill_color='aqua')
        # m.fillcontinents(color='gray',lake_color='white',zorder=2)
        # m.fillcontinents(color='gray',lake_color=None, zorder=1)
        # m.fillcontinents(color='coral',lake_color='white')
        m.drawlsmask(land_color='gray', ocean_color='white', lakes=True)
        # m.drawcountries(zorder=2)
        # m.drawcoastlines()
        
        # m.drawmapboundary(fill_color='aqua')
        # m.drawparallels(np.arange(-90., 91., 15.))
        # m.drawmeridians(np.arange(-180., 181., 30.))
        # m.shadedrelief()
        # print(f"LON_LAT: {lon[0], lat[0]}")
        lons, lats = m(lon, lat, 
                #        inverse=True
                       )
        # print(lons[0], lats[0])
        if plot_scatter:
                # print('Plot Scatter')
                # print(lon)
                # print(lat)
                x, y = m(lon, lat, 
                        #        inverse=True
                               )        
                m.scatter(x, y, 
                          marker = 'o', 
                          color='r', 
                          zorder=1, 
                          label=1,
                          s=zoom+1
                          )
                # plt.title(names)
        # if projection=='merc':
        #     plt.title("Mercator Projection")
        # #     plt.show()
        # elif projection=='npstere':
        #     plt.title("North Pole Projection")
        # #     plt.show()
        if img_write==True:
                os.makedirs(f'./tiles/{projection}/{tile}/', exist_ok=True)
                plt.savefig(
                        f'./tiles/{projection}/{tile}/{slice_tile}.png', 
                        bbox_inches='tight', 
                        pad_inches = 0,
                        dpi=100
                        )
                
                plt.close()
                plt.clf()
                
        return m
def draw_map(
        lats_min=-80,
        lats_max=80,
        lons_min=-180,
        lons_max=180,
        resolution='i',
        projection='merc',
        zoom=1,
        lat=[0],
        lon=[0],
        tile='0',
        slice_tile='0',
        img_write=False,
        plot_scatter=False,
        names=[]
        ):        
        # fig = plt.gcf()
        # fig.get_size_inches()
        # fig.dpi
        
        # plt.rcParams['figure.dpi'] = 100
        # plt.rcParams['figure.figsize'] = (2.56 * zoom, 2.56 * zoom)
        
        plt.subplots_adjust(top = 1, bottom = 0, right = 1, left = 0, 
            hspace = 0, wspace = 0)
        plt.margins(0,0)
        if projection=='merc':
                m = Basemap(
                        projection=projection,
                        llcrnrlat=lats_min,
                        urcrnrlat=lats_max,
                        llcrnrlon=lons_min,
                        urcrnrlon=lons_max,
                        lat_ts=0,
                        resolution=resolution,
                        )
                return _extracted_from_draw_map_19(m, 
                                                   projection, 
                                                   zoom,
                                                   lat, lon,
                                                   tile, slice_tile, 
                                                   img_write=img_write,
                                                   plot_scatter=plot_scatter,
                                                   names=names
                                                   )
        elif projection=='npstere':
                m = Basemap(
                        projection='npstere',
                        resolution=resolution,
                        lon_0=0.,
                        boundinglat=10,                       
                )
                return _extracted_from_draw_map_19(m, projection, zoom,
                                                   lat, lon,
                                                   tile, slice_tile, 
                                                   img_write=img_write,
                                                   plot_scatter=plot_scatter,
                                                   names=names
                                                   )

In [10]:
int(320/8), int(160/8)

(40, 20)

In [21]:
tile = 3
if tile == 3:
    step_x, step_y = int(360/8), int(160/8)
    step_y = 80
    step_x = 20
step_x, step_y

(20, 80)

In [22]:
actual_st['lat'].max(), actual_st['lat'].min()

(70.317, 38.45)

In [23]:
actual_st['lon'].max(), actual_st['lon'].min()

(78.7, 19.5)

In [24]:
resolution = 'h'
latmin = 80.

In [25]:
i=0
for tile_x in tqdm(range(0, 361, step_x)):
# for tile_y in reversed(range(0, 80, step_y)):
    # print('*******************************************')
    df = actual_st[(actual_st['lon'] >= tile_x)\
        & (actual_st['lon'] < tile_x+step_x)\
            & (actual_st['lat'] >= 0)\
                & (actual_st['lat'] < latmin)]
    # print(f"df.shape={df.shape}")
    st_lats_lons = df.loc[:, ['lat', 'lon']].values
    # print(f"st_lats_lons={st_lats_lons.shape}")
    names = df['name'].to_list()
    # print(names)
    # print(f"{i} : lon0={tile_x} lon1={tile_x+step_x} lat0={tile_y} lat1={tile_y+step_y}")
    if st_lats_lons.shape[0] > 0:
        # print(st_lats_lons.shape)
        map_ = draw_map(
            lats_min = int(0),
            lats_max = int(latmin),
            lons_min = int(tile_x),
            lons_max = int(step_x + tile_x),
            projection='merc',
            resolution = resolution,
            zoom=1,
            lat=st_lats_lons[:,0],
            lon=st_lats_lons[:,1],
            tile=str(tile),
            slice_tile=str(i),
            img_write=True,
                plot_scatter=True,
                # names=names
            )
    elif st_lats_lons.shape[0] == 0:
        map_ = draw_map(
            lats_min = int(0.),
            lats_max = int(latmin),
            lons_min = int(tile_x),
            lons_max = int(step_x + tile_x),
            projection='merc',
            resolution = resolution,
            zoom=1,
            lat=[],
            lon=[],
            tile=str(tile),
            slice_tile=str(i),
            img_write=True,
            plot_scatter=False,
        )
    i += 1

del map_ 
del st_lats_lons
plt.clf()
plt.close()

100%|██████████| 19/19 [06:36<00:00, 20.85s/it]


In [26]:
import pathlib
tiles_files = sorted([str(i) for i in pathlib.Path("./tiles/").glob('**/*.png')])
for i in tiles_files:
    img = Image.open(i)
    print(img.size)

(171, 1200)
(171, 1200)
(171, 1200)
(171, 1200)
(171, 1200)
(171, 1200)
(171, 1200)
(171, 1200)
(171, 1200)
(171, 1200)
(171, 1200)
(171, 1200)
(171, 1200)
(171, 1200)
(171, 1200)
(171, 1200)
(171, 1200)
(171, 1200)
(171, 1200)


In [19]:
1200/480

2.5

In [20]:
343/137

2.5036496350364965